# Maximum likelihood estimation with Kalman filter and automatic differentiation

### This notebook shows how to use `Kalman.jl` together with `ForwardDiff` to estimate the filter coefficients.


In [44]:
using Pkg
!haskey(Pkg.installed(), "GaussianDistributions") && Pkg.clone("https://github.com/mschauer/GaussianDistributions.jl")
!haskey(Pkg.installed(), "Kalman") && Pkg.clone("https://github.com/mschauer/Kalman.jl");

In [45]:
using Kalman
using StaticArrays
using Random, LinearAlgebra
using GaussianDistributions
using DynamicIterators
using Trajectories

In [46]:
using ForwardDiff
using ForwardDiff: derivative, Dual

## Define a parametrized linear statespace model with parameter `θ`

In [56]:
x0 = [1., 0.]
P0 = Matrix(1.0I, 2, 2)

Φ(θ) = [0.8 θ/2; -0.1 0.8]
b = zeros(2)
Q = [0.2 0.0; 0.0 1.0]

yshadow = [0.0]
H = [1.0 0.0]
R = Matrix(0.2I, 1, 1)

E(θ) = LinearEvolution(Φ(θ), Gaussian(b, Q))
Obs = LinearObservationModel(H, R)
M(θ) = LinearStateSpaceModel(E(θ), Obs)


O(θ) = LinearObservation(E(θ), H, R)


O (generic function with 1 method)

## Generate test data with `θ0 = 1.0`

In [57]:
θ0 = 1.0
M0 = M(θ0)

LinearStateSpaceModel{LinearEvolution{Array{Float64,2},Gaussian{Array{Float64,1},Array{Float64,2}}},LinearObservationModel{Array{Float64,2},Array{Float64,2}}}(LinearEvolution{Array{Float64,2},Gaussian{Array{Float64,1},Array{Float64,2}}}([0.8 0.5; -0.1 0.8], Gaussian{Array{Float64,1},Array{Float64,2}}([0.0, 0.0], [0.2 0.0; 0.0 1.0])), LinearObservationModel{Array{Float64,2},Array{Float64,2}}([1.0 0.0], [0.2]))

In [58]:
Random.seed!(11)
X = trace(DynamicIterators.Sampled(LinearEvolution(Φ(θ0), Gaussian(b, Q))), 0 => x0, endtime(10000))

Y = collect(t=>SVector{1}(y) for (t, (x,y)) in pairs(X))


10001-element Array{Pair{Int64,SArray{Tuple{1},Float64,1,1}},1}:
     0 => [0.0]      
     1 => [0.201635] 
     2 => [2.04921]  
     3 => [1.30766]  
     4 => [2.26442]  
     5 => [1.61125]  
     6 => [1.46419]  
     7 => [0.416179] 
     8 => [0.339188] 
     9 => [-1.05974] 
    10 => [-0.981097]
    11 => [-1.7979]  
    12 => [-2.95037] 
       ⋮             
  9989 => [-1.32969] 
  9990 => [-2.58169] 
  9991 => [-3.75926] 
  9992 => [-3.24176] 
  9993 => [-1.30553] 
  9994 => [2.60522]  
  9995 => [1.16561]  
  9996 => [1.16594]  
  9997 => [1.36326]  
  9998 => [1.41969]  
  9999 => [1.21885]  
 10000 => [1.74465]  

In [59]:
kalmanfilter(O(θ0), 0 => Gaussian(x0, P0), Y)[2]

-14716.974601164988

## Define iterator that sums the marginal log likelihood of the data

In [60]:
function filter(O, prior, Y) # Kalman filter using iteration, could be easily adapted to an only version
    ϕ = iterate(Y) 
    ϕ === nothing && error("no observations")
    y, ystate = ϕ

    ϕ = dyniterate(O, Start(Kalman.Filter(prior, 0.0)), y)
    ϕ === nothing && error("no observations")
    (t, u), state = ϕ

    X = trajectory((t => u[1],))
    while true
        ϕ = iterate(Y, ystate)
        ϕ === nothing && break
        y, ystate = ϕ

        ϕ = dyniterate(O, state, y) # dyniterate is an iteration protocol which allows iterators depends on an input y 
        ϕ === nothing && break
        (t, u), state = ϕ
    end
    ll = u[3] # likelihood
end


filter (generic function with 2 methods)

In [61]:
target(θ) = filter(O(θ[]), 0 => Gaussian(x0, P0), Y)

(-14716.974601164988, -14730.792632014904, -14735.56554895877)

## Fit `θ` using Newton with automatic derivatives

In [62]:
θ = [0.7]
println(θ)
for it in 1:10
    θold = θ
    θ = θ .- inv(ForwardDiff.hessian(target, θ))*ForwardDiff.derivative(target, θ[])
    θold == θ && break    
    println(θ)
end
println("Maximum llikelihood estimate: θ = $θ")

[0.7]
[0.906379]
[0.981415]
[0.988858]
[0.988922]
[0.988922]
[0.988922]
[0.988922]
[0.988922]
[0.988922]
[0.988922]
Maximum llikelihood estimate: θ = [0.988922]
